# Reading Data from csv file and doing basic data manipulation

In [ ]:
import pandas as pd
df = pd.read_csv("1.csv", index_col=0)

## Get data for individual Trips

In [ ]:
for tripID in set(df['tripID'].values):
    df_trip = df[df['tripID']==tripID]

## Converting Raw ECompass data to engineering units

In [ ]:
import numpy as np
# Convert Accelerometer reading to g units, 8bit data with a range of +-2g #
def convert_acc(x):
    x = int(x,16)
    if x>127:
        x = x-256
    return np.float64(x*2/128)

# Convert Magnetometer reading to uT units, 16bit data with a range of +-1200uT #
def convert_mag(x):
    x = int(x,16)
    if x>32767:
        x = x-65536
    return np.float64(x*0.0366)


# Convert a single row of Accelerometer data to engineering units and return a dict #
def convert_acc_row(row):
    # Initially the data was gathered without magnetometer, so check length for identification #
    data_dict = {'x': [], 'y': [], 'z': []}
    if len(row) == 150:
        for i in range(0, len(row), 6):
            data_dict['x'].append(convert_acc(row[i:i+2]))
            data_dict['y'].append(convert_acc(row[i+2:i+4]))
            data_dict['z'].append(convert_acc(row[i+4:i+6]))
    return data_dict


# Convert a single row of Accelerometer and Magnetometer data to engineering units and return a df #
def convert_acc_mag_row(row):
    # Initially the data was gathered without magnetometer, so check length for identification #
    data_dict = {'x': [], 'y': [], 'z': []}
    if len(row) == 162:
        data_dict['mx'] = convert_mag(row[:4])
        data_dict['my'] = convert_mag(row[4:8])
        data_dict['mz'] = convert_mag(row[8:12])
        row = row[12:]
        for i in range(0, len(row), 6):
            data_dict['x'].append(convert_acc(row[i:i+2]))
            data_dict['y'].append(convert_acc(row[i+2:i+4]))
            data_dict['z'].append(convert_acc(row[i+4:i+6]))
    return data_list

df_trip['accData'] = df_trip['accData'].apply(convert_acc_row)
# Drop the values for first second as the data is not collected completely #
df_trip = df_trip.reset_index(drop=True)
df_trip = df_trip.drop(0).reset_index(drop=True)

### Calculating Rotation Matrix for vehicle

In [ ]:
# We need data for the device when the vehicle is static #
[mx,my,mz] = df_trip[['mx', 'my', 'mz']][df_trip['speed'] <= 1].mean().tolist()
[x,y,z] = df_trip['accData'][df_trip['speed'] <= 1].iloc[0].mean().tolist()

import math
phi = math.atan(y/z)
theta = math.atan(-x/(y*math.sin(phi)+z*math.cos(phi)))
psi = math.atan((mz*math.sin(phi) - my*math.cos(phi))/(mx*math.cos(theta) + my*math.sin(theta)*math.sin(phi) + mz*math.sin(theta)*math.cos(phi)))

Rx = np.array([[1,0,0], [0,math.cos(phi),math.sin(phi)], [0,-math.sin(phi),math.cos(phi)]])
Ry = np.array([[math.cos(theta),0,-math.sin(theta)], [0,1,0], [math.sin(theta),0,math.cos(theta)]])
Rz = np.array([[math.cos(phi),math.sin(phi),0], [-math.sin(phi),math.cos(phi),0], [0,0,1]])

R = Rx.dot(Ry).dot(Rz)
R = np.linalg.inv(R)

### Rotate Accelerometer values

In [ ]:
def rotate_acc(row):
    A = np.array([[row['ax']], [row['ay']], [row['az']]])
    B = np.dot(R,A)
    return B[:,0]

acc_sample = df_trip['accData'].iloc[0]
acc_sample.apply(rotate_acc, axis=1)

## Storing DataFrame in hdf file

In [ ]:
df.to_hdf("data.hdf5", "first")